In [35]:
import glob
import csv
import pandas
import os
import re
import sys
import time
import datetime
import pprint as pp

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
from openpyxl import load_workbook

import configparser 
config = configparser.ConfigParser()
config.read('config.ini')
ip = config['DEFAULT']['IP']
port = config['DEFAULT']['MongoDB-Port']
contain_string = config['DEFAULT']['Contain-String']

from pymongo import MongoClient
client = MongoClient(ip, int(port))

In [36]:
db_twitter = client["Twitter"]
collections_twitter = db_twitter.collection_names()
pp.pprint(collections_twitter)

['twitter-richard-2018',
 'twitter-richard-2015',
 '2019_W04_Twitter_Other',
 '2019_W08_Twitter_Other',
 '2019_W08_Twitter_Australia',
 'twitter-richard-2017',
 '2019_W03_Twitter_Australia',
 '2019_W06_Twitter_Australia',
 '2018_W51_Twitter_Australia',
 '2019_W05_Twitter_Australia',
 '2019_W10_Twitter_Australia',
 '2019_W13_Twitter_Other',
 '2019_W12_Twitter_Other',
 '2019_W10_Twitter_Other',
 '2019_W14_Twitter_Other',
 'twitter-richard-2016',
 '2019_W06_Twitter_Other',
 '2019_W01_Twitter_Other',
 'twitter-richard-2014',
 '2019_W05_Twitter_Other',
 '2018_W52_Twitter_Other',
 '2019_W07_Twitter_Other',
 '2019_W12_Twitter_Australia',
 '2019_W07_Twitter_Australia',
 '2019_W01_Twitter_Australia',
 '2019_W02_Twitter_Australia',
 '2019_W02_Twitter_Other',
 '2019_W13_Twitter_Australia',
 '2019_W09_Twitter_Australia',
 '2018_W52_Twitter_Australia',
 '2019_W_Twitter_Other',
 '2018_W51_Twitter_Other',
 '2019_W14_Twitter_Australia',
 '2019_W11_Twitter_Australia',
 '2019_W03_Twitter_Other',
 '2019_

In [37]:
# get current year
current_timestamp = int(time.time() * 1000)
current_year = int(datetime.datetime.now().year)
print("current year : " + str(current_year))

# get current week
current_week = int((current_timestamp - 1546214400000)/1000/604800)+1
print("current week : " + str(current_week))

current year : 2019
current week : 14


In [38]:
dic_collection = {}
for i in collections_twitter:
    if i.startswith("20") and contain_string in i:
        year = i[0:4]
        week = re.search('_(.+?)_', i).group(1)[1:]
        if int(year) < current_year:
            dic_collection[i] = "{:}".format(db_twitter[i].find({}).count())
        else:
            try:
                if int(week) < current_week:
                    dic_collection[i] = "{:}".format(db_twitter[i].find({}).count())
            except: pass

for key in sorted(dic_collection):
    print("%s: %s" % (key, dic_collection[key]))

2018_W51_Twitter_Australia: 20471
2018_W51_Twitter_Other: 8978243
2018_W52_Twitter_Australia: 38065
2018_W52_Twitter_Other: 16940708
2019_W01_Twitter_Australia: 40880
2019_W01_Twitter_Other: 16994959
2019_W02_Twitter_Australia: 37645
2019_W02_Twitter_Other: 14474627
2019_W03_Twitter_Australia: 52348
2019_W03_Twitter_Other: 20416992
2019_W04_Twitter_Australia: 59625
2019_W04_Twitter_Other: 23417927
2019_W05_Twitter_Australia: 61617
2019_W05_Twitter_Other: 23917345
2019_W06_Twitter_Australia: 61435
2019_W06_Twitter_Other: 24176122
2019_W07_Twitter_Australia: 28017
2019_W07_Twitter_Other: 10484648
2019_W08_Twitter_Australia: 60035
2019_W08_Twitter_Other: 23446630
2019_W09_Twitter_Australia: 58316
2019_W09_Twitter_Other: 22293795
2019_W10_Twitter_Australia: 40767
2019_W10_Twitter_Other: 15941350
2019_W11_Twitter_Australia: 68593
2019_W11_Twitter_Other: 25345811
2019_W12_Twitter_Australia: 50447
2019_W12_Twitter_Other: 18090490
2019_W13_Twitter_Australia: 62926
2019_W13_Twitter_Other: 24379

# Extract a hashtag_city report from individual collections

In [39]:
# read city, state, country from csv file
colnames = ['city', 'state', 'country']
au_data = pandas.read_csv('supporting-files/au.csv', names=colnames, encoding="ISO-8859-1")
world_data = pandas.read_csv('supporting-files/world-cities.csv', names=colnames, encoding="ISO-8859-1")
pp.pprint ('AU Cities: {}'.format(len(au_data)))
pp.pprint ('International Cities {}'.format(len(world_data)))


'AU Cities: 220'
'International Cities 22815'


In [40]:
pipeline = [
    {"$match": { "entities.hashtags": {"$exists":True,"$ne":[]}}},
    {"$match": { "lang" : "en"}},
    { "$group": {
        "_id": {
            "hashtags": "$entities.hashtags",
            "location": "$user.location"},
        "count": { "$sum": 1 },
        }
    }
]

In [41]:
# get list from csv file
def get_list_from_csv(data):
    ci = data.city.tolist()
    city = ci[1:]
    st = data.state.tolist()
    state = st[1:]
    co = data.country.tolist()
    country = co[1:]
    return city,state,country

In [42]:
# create foler if not exist
def create_folder():
    folder = "output/hashtag_city/"
    if not os.path.exists(folder):
        os.makedirs(folder)
    return folder

In [43]:
# delete existed collection from the list dic_collection
def delete_collection(folder,dic_collection):
    for input_file in glob.glob(os.path.join(folder,'*.csv')):
        collection_name = re.search('{(.+?)}', input_file).group(1)
        print("Existed collection: " + collection_name)
        del dic_collection[collection_name]
    return dic_collection

In [44]:
# method to add count number
def count(dc,data,exist,c):
    dc += data["count"]
    exist = 1
    c+=1
    return dc,exist,c

In [45]:
# append to dictionary
def append_dic(data_format,exist,c,dic):
    data_format.append(dic)
    exist = 1
    c+=1
    return data_format,exist,c

In [46]:
# check if exists then add count number
def check_count(data_format, x,text,s_geo,exist,c,data):
    for d in data_format:
        s0 = d["id"].split(",")[0]
        s1 = s = d["id"].split(",")[1]
        if(x in d["id"]) and (text in d["hashtag"]) and (s_geo == "city"):
            d["count"],exist,c = count(d["count"],data,exist,c)
            break
        elif (s0 == "null") and (x in d["id"]) and (text in d["hashtag"]) and (s_geo == "state"):
            d["count"],exist,c = count(d["count"],data,exist,c)
            break
        elif (s1 == "null") and (x in d["id"]) and (text in d["hashtag"])and (s_geo == "country"):
            d["count"],exist,c = count(d["count"],data,exist,c)
            break
    return data_format,exist,c

In [47]:
# calculate running time
def calculate_time(start_time, t):
    current_time = time.time()
    duration = current_time - start_time
    if (duration/3600) >= (t+1):
        t += 1
        print("The program is still running, already run for about "+ str(t) + " hours.")
    return t

In [48]:
# method for each loop
def floop(s_geo,geo,data,data_format,text,state,country,exist,c,start_time,t):
    for x in geo: 
        if isinstance(x,str):
            if s_geo == "city":
                loc = data["_id"]["location"].lower().replace("australia","")
            else:
                loc = data["_id"]["location"].lower()
            if x.lower() in loc:
                index = geo.index(x)
    
                if s_geo == "city":
                    sid = x + ',' + str(state[index]) + "," + str(country[index])
                if s_geo == "state":
                    sid = "null" + "," + x + "," + str(country[index])
                if s_geo == "country":
                    sid = "null" + "," + "null" + ',' + x
                dic = {"id":sid,"hashtag":text,"count":data["count"]}
                
                if len(data_format)>0:
                    data_format,exist,c = check_count(data_format, x,text,s_geo,exist,c,data)
                    if exist == 0:
                        data_format,exist,c = append_dic(data_format,exist,c,dic)
                else: 
                    data_format,exist,c = append_dic(data_format,exist,c,dic)
                    
                #print("No." + str(c) + s_geo + ": " + x)
                
                # print every hour if it's still running
                t = calculate_time(start_time, t)
                break
   
    return data_format,exist,c,t

In [49]:
# write csv file
def write_csv(collection,total_tweet,tweets_htag,data_format):
    
    file_name = "output/hashtag_city/{" + collection + "}_hashtag_city.csv"
    with open(file_name, 'w') as f:
        # header
        f.write('hashtag,city,state,country,number of tweets,tweets_htag,tweets_nohtag\n')
            
        tweets_nohtag = int(total_tweet)-tweets_htag

        for data in data_format:
            city = data['id'].split(",")[0]
            state = data['id'].split(",")[1]
            country = data['id'].split(",")[2]
            if "Australia" in collection:
                if state == 'Victoria':
                    state = 'VIC'

            line = data["hashtag"] + ',' + city + ',' + state + ',' +  country + ',' + \
            str(data['count']) + ','+ str(tweets_htag)+ ',' +str(tweets_nohtag)+ '\n'
            
            f.write(line)                

In [50]:
csv_columns = ['hashtag','city','state','country','count']

#create folder if not exist
folder = create_folder()
    
# delete existed collection from the list dic_collection
dic_collection = delete_collection(folder,dic_collection)


for collection in sorted(dic_collection):
    
    start_time = time.time()
    t = 0
 
    total_tweet = dic_collection[collection]
    if "Australia" in collection:     
        city, state, country = get_list_from_csv(au_data)
    elif "Other" in collection:
        city, state, country = get_list_from_csv(world_data)

    print("-----------------------\n")
    print("Processing on collection: " + collection)

    data_format = []
    
    print("Aggregating...")
    data_list = list(db_twitter[collection].aggregate(pipeline,allowDiskUse=True))
    print("Aggragation finished.")
    c = 0 
    tweets_htag = 0
    
    if len(data_list) > 0 : 
        for data in data_list:
            tweets_htag += data["count"]
            for htag in data["_id"]["hashtags"]:
                # get hashtag
                text = htag["text"].lower()
                # check if it is in English
                if(re.match("^[a-zA-Z0-9]*$",text)):
                    exist = 0
                    # check if the location is null
                    if data["_id"]["location"] is not None: 
                        s_geo = "city"
                        data_format,exist,c,t = floop(s_geo,city,data,data_format,text,state,country,exist,c,start_time,t)
                        if exist == 0:
                            s_geo = "state"
                            data_format,exist,c,t = floop(s_geo,state,data,data_format,text,state,country,exist,c,start_time,t)
                        if exist == 0:
                            s_geo = "country"
                            data_format,exist,c,t = floop(s_geo,country,data,data_format,text,state,country,exist,c,start_time,t)


        print("hashtag list is finished")

        write_csv(collection,total_tweet,tweets_htag,data_format)

        print ("csv file for collection " + collection + ' is finished.')
        print("-----------------------\n")

-----------------------

Processing on collection: 2018_W51_Twitter_Australia
Aggregating...
Aggragation finished.
hashtag list is finished
csv file for collection 2018_W51_Twitter_Australia is finished.
-----------------------

-----------------------

Processing on collection: 2018_W51_Twitter_Other
Aggregating...


KeyboardInterrupt: 